In [1]:
from ore_algebra import *

from ore_algebra.analytic.differential_operator import DifferentialOperator
from ore_algebra.analytic.local_solutions import (log_series, LocalBasisMapper,
                                                  simplify_exponent, LogMonomial)
from ore_algebra.analytic.path import Point

In [2]:
import itertools as it

def group_by_module(roots):
    '''
    Returns an iterator over groups of roots, grouped by increasing module.
    Assumes `roots` to be an iterable over 2-tuples of the form (root, multiplicity).
    '''
    def root_module(t):
        return abs(t[0])
    roots.sort(key=root_module)
    return it.groupby(roots, key=root_module)

In [3]:
def is_regular_singular_point(point, leading_mult, var, op):
    '''
    Checks wether a point is a regular singular point of a differential operator.
    `leading_mult` is the multiplicity of `point` as a root
    of the leading coefficient of `op`.
    '''
    r = op.order()
    for i, poly in op.dict().items(): # 0 -> p_0, ...
        if r - i < leading_mult \
                and not ((var - point)^(leading_mult - (r - i))).divides(poly):
            return False
    return True

In [4]:
def my_expansions(op, point, order=None, ring=None):
    mypoint = Point(point, op)
    dop = DifferentialOperator(op)
    ldop = dop.shift(mypoint)
    if order is None:
        ind = ldop.indicial_polynomial(ldop.base_ring().gen())
        order = max(dop.order(), ind.dispersion()) + 3
    class Mapper(LocalBasisMapper):
        def fun(self, ini):
            return log_series(ini, self.shifted_bwrec, order)
    sols = Mapper(ldop).run()
    x = SR.var(dop.base_ring().variable_name())
    dx = x if point.is_zero() else x.add(-point, hold=True)
    if ring is None:
        cm = get_coercion_model()
        ring = cm.common_parent(
                dop.base_ring().base_ring(),
                mypoint.value.parent(),
                *(sol.leftmost for sol in sols))
    return [[(c / ZZ(k).factorial() * (-point)^sol.leftmost, point, sol.leftmost + n, k)
                    for n, vec in enumerate(sol.value)
                    for k, c in reversed(list(enumerate(vec)))]
            for sol in sols]

##### Prop 2.17 (p66) :
Si
$F(z) \sim C {(1 - \frac{z}{\zeta})}^\alpha \log^m (1 - \frac{z}{\zeta})$ avec $\alpha \not\in \mathbb{N}$, alors
$f_n \sim C \zeta^{-n} \frac{n^{-\alpha - 1}}{\Gamma(-\alpha)} \log^m(n)$.

In [5]:
class GeneralMonomial:
    def __init__(self, c, zeta, alpha, m):
        # TODO assert alpha not in NN et gerer différemment
        self.c = c
        self.zeta = zeta
        self.alpha = alpha
        self.m = m
        self.asymp_c = c / gamma(-alpha)

    def is_singular(self):
        return self.alpha < 0 or self.m != 0

    def __repr__(self):
        if self.m == 0:
            return f'{self.asymp_c} * {1/self.zeta}^n * n^{-self.alpha - 1}'
        return f'{self.asymp_c} * {1/self.zeta}^n * n^{-self.alpha - 1} * log^{self.m}(n)'

    def asymptotic_symbolic(self):
        n = var('n')
        return self.c / gamma(-self.alpha) * self.zeta^n * n^(-self.alpha - 1) * log(n)^self.m

    def __cmp__(self, other):
        biggest = None
        if abs(self.zeta) != abs(other.zeta):
            biggest = self if self.zeta < other.zeta else other  # le plus petit |zeta| gagne
        elif self.alpha != other.alpha:
            biggest = self if self.alpha < other.alpha else other  # le plus petit alpha gagne
        elif self.m != other.m:
            biggest = self if self.m > other.m else other  # le plus grand m gagne
        if biggest is None or not biggest.c.is_nonzero():
            return 0
        return 1 if biggest is self else -1
    
    def __le__(self, other):
        return 0 if self.__cmp__(other) == 1 else 1
    
    def __lt__(self, other):
        return 1 if self.__cmp__(other) == -1 else 0

In [6]:
def compute_initial_decomp(op, first_coefficients):
    def is_singular_func(func):
        for coeff, mon in func:
            if coeff != 0 and (mon.n not in NN or mon.k != 0):
                return True
        return False
    
    def extract_coeff(fsum, n):
        return next(t[0] for t in fsum if t[1].n == n and t[1].k == 0)

    basis = op.local_basis_expansions(0, order=op.order())

    sing_vect = [is_singular_func(func) for func in basis]
    nb_of_non_sing = len(basis) - sum(sing_vect)
    mat = matrix([
        [
            extract_coeff(fsum, n)
            for fsum, is_sing in zip(basis, sing_vect) if not is_sing
        ]
        for n in range(nb_of_non_sing)
    ])
    non_sing_coeffs = mat.inverse() * vector(first_coefficients[:nb_of_non_sing])
    
    it_non_sing_coeffs = iter(non_sing_coeffs)
    decomp = [0 if is_sing else next(it_non_sing_coeffs)
              for is_sing in sing_vect]
    
    return basis, decomp

In [7]:
EPS = 1e-20
ORDER = 6

def handle_root(op, root, ini):
    trans_matrix = op.numerical_transition_matrix([0, root], eps=EPS)
    coeffs_in_local_basis = trans_matrix * vector(ini)

    local_expansions = my_expansions(op, root, order=ORDER)

    new_monomials = [GeneralMonomial(lambda_i * c, zeta, alpha, m)
                     for terms in local_expansions
                     for lambda_i, (c, zeta, alpha, m) in zip(coeffs_in_local_basis, terms)]
    
    return new_monomials

In [8]:
Pols.<z> = PolynomialRing(QQ)
Diff.<Dz> = OreAlgebra(Pols)

op = (4*z^2 - z)*Dz^2 + (14*z - 2)*Dz + 6

basis_in_0, ini = compute_initial_decomp(op, [1, 3])
print(ini)

[0, 1]


In [9]:
roots = op.leading_coefficient().roots(QQbar)
# TODO check 0 is reg or sing reg

found_a_sing = False
got_problem = False
all_monomials = []
for module, roots in group_by_module(roots):
    roots = list(roots)
    if module == 0:
        continue

    for root, multiplicity in roots:

        if not is_regular_singular_point(root, multiplicity, z, op):
                print('Got a non regular singular point. Stoping here')
                got_problem = True
                break

        new_monomials = handle_root(op, root, ini)
        all_monomials += new_monomials

        if any(mon.is_singular() and mon.c.is_nonzero()
               for mon in new_monomials):
            found_a_sing = True
        
    if got_problem:
        break
    if found_a_sing:
        all_monomials.sort(reverse=True)
        print(sum((mon.asymptotic_symbolic() for mon in all_monomials), start=0))
        break

([2.00000000000000000000 +/- 1.70e-21])*(1/4)^n/(sqrt(pi)*sqrt(n)) + ([8.00000000000000000000 +/- 6.78e-21]*I)*(1/4)^n/(sqrt(pi)*n^(3/2))


In [10]:
def test_is_regular_singular_point():
    Pols.<z> = PolynomialRing(QQ)
    Diff.<Dz> = OreAlgebra(Pols)

    op = (4*z^2 - z) * Dz^2 + (14 * z - 2) * Dz + 6
    assert is_regular_singular_point(0, 1, z, op) == True
    assert is_regular_singular_point(1/4, 1, z, op) == True
    
    op2 = (4*z^3 - z^2) * Dz^2 + (14 * z - 2) * Dz + 6
    assert is_regular_singular_point(0, 2, z, op2) == False
    assert is_regular_singular_point(1/4, 1, z, op2) == True
    
    op3 = (z)^4 * (z-1)^5 * Dz^3 + (z^2) * (z - 1) * Dz
    assert is_regular_singular_point(0, 4, z, op3) == True
    assert is_regular_singular_point(1, 5, z, op3) == False
    
    op4 = (z-1/4)^8 * (z-1/5)^12 * Dz^9 + (z - 1/4)^4 * Dz^5 + 1
    assert is_regular_singular_point(1/4, 8, z, op4) == True
    assert is_regular_singular_point(1/5, 12, z, op4) == False

test_is_regular_singular_point()